In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,8):
    print 'scrapping page : ', i
    url = 'http://ieatishootipost.sg/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h3', {'class' : 'entry-title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('div', {'class' : 'article-date'}).text[11:]
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "article-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  http://ieatishootipost.sg/stir-fried-potatoes-pork-simple-homecooked-meals/
cleaned detail snippet :  potatoes and pork go so well together if you have never tried this dish bef
href :  http://ieatishootipost.sg/kyoaji-singaporean-japanese-food/
cleaned detail snippet :  the california roll was created over five decades ago in los angeles by che
href :  http://ieatishootipost.sg/cabbage-yam-brown-rice-tefal-spherical-bowl-induction-rice-cooker-recipe/
cleaned detail snippet :  here is my recipe for cabbage yam brown rice recipe which can all be made i
href :  http://ieatishootipost.sg/springleaf-prata-railway-mall-happy-pratamen/
cleaned detail snippet :  what is the fundamental difference between a prata and a pizza one ingredie
href :  http://ieatishootipost.sg/wee-family-seafood-zi-char-wine-pairing/
cleaned detail snippet :  we might be worried about the disappearance of certain hawker dishes like c
href :  http://ieatishootipost.sg/jia-xiang-nasi-lemak-

cleaned detail snippet :  it is amazing how the hdb bakery has changed over the years i still remembe
href :  http://ieatishootipost.sg/eat-first-%e9%a3%9f%e4%b9%8b%e7%82%ba%e9%ae%ae-tale-two-brothers/
cleaned detail snippet :  once upon a time there was a man who ran a very successful restaurant servi
href :  http://ieatishootipost.sg/charlies-peranakan-food-is-back/
cleaned detail snippet :  you really cant keep a good cook out of the kitchen for too long when i las
href :  http://ieatishootipost.sg/tomato-egg-recipe-10-min-meals/
cleaned detail snippet :  lots of people have requested quick and easy recipes that they can put toge
scrapping page :  5
href :  http://ieatishootipost.sg/yong-he-bak-chor-seafood-noodles-surf-and-turf/
cleaned detail snippet :  bak chor mee has always been an understated singaporean dish at least it wa
href :  http://ieatishootipost.sg/si-yang-steam-soup/
cleaned detail snippet :  it really isnt easy being a hawker imagine that you have been working hard 

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.515434980392  seconds
85
85


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 4.01454114914  seconds
85
85


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
80,Top 10 Singapore Food Finds of 2016!,This year is quite special for me as it marks ...,"December 28, 2016",find,special mark year eating shooting posting adve...
81,Easy Recipe Ideas for Christmas Leftovers,The most wonderful time of the year is upon us...,"December 22, 2016",easy recipe idea christmas leftover,wonderful looking forward spending family frie...
82,Hokkaido Izakaya: Yummy Hokkaido Food,"With its harsh winters, remote location and pr...","December 20, 2016",hokkaido izakaya yummy hokkaido,harsh winter remote location pristine water ho...
83,Leslie’s Fried Chicken (LFC) Recipe,"In this post, I am going to present to you the...","December 16, 2016",leslies fried chicken lfc recipe,post going present final recipe fried chicken ...
84,Wah Kee Prawn Mee: The Next Generation,Singapore’s hawker food scene has really comin...,"December 13, 2016",wah kee prawn mee generation,singapore hawker scene coming age late seeing ...


In [8]:
len(df)

85

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

85

In [11]:
x = df.iloc[3,1]
print x

What is the fundamental difference between a prata and a pizza? One ingredient:  Yeast The other ingredients for the dough is essentially the same:  Plain flour, water, salt, sugar and oil.  All you need to change a prata dough into a pizza dough is to add yeast to the mix and allow it to proof! So, why is it that people are happy paying $20 for a pizza, but would think twice about paying half of that for a murtabak with the same ingredients? That is the kind of culinary prejudice that is working against purveyors of local cuisine.  The kind of prejudice which my friend Gunalan, the owner of Springleaf Prata Place had to face when he started to introduce his creative pratas many years ago! When I first met Guna, he had already been doing all sorts of things with prata.  He started with a chicken floss prata and went on to create chocolate prata, red bean prata and the like.  But at that time, he was hard pressed to sell anything above $5. The breakthrough came when we created the Ultim

In [12]:
# pickle dataframe
df.to_pickle('ieatishoot.pickle')

In [13]:
wd.quit()